In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

from trial_class import *
from experiment_class import Experiment

import numpy as np
import pandas as pd
import tdt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from scipy.signal import butter, filtfilt
from sklearn.linear_model import LinearRegression
from sp_extension import *

In [2]:
experiment_path = r"/Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core"
csv_base_path = r"/Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core_csvs"
brain_region = '#15616F'

# NAc: #15616F
# mPFC: #FFAF00

In [3]:
# groups csv + experiment data into one variable
experiment = Experiment(experiment_path, csv_base_path)

# batch process the data, removing the specified time segments for subjects
experiment.default_batch_process()

Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core/p5-240826-091418/Notes.txt
read from t=0s to t=790.95s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core/nn2-250117-085631/Notes.txt
read from t=0s to t=814.05s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core/p2-240523-081105/Notes.txt
read from t=0s to t=788.65s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core/nn4-250118-094351/Notes.txt
read from t=0s to t=795.95s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core/nn1-250117-081652/Notes.txt
read from t=0s to t=870.75s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all

In [ ]:
bout_definitions = [
    {
        'prefix': 'Subject',
        'introduced': 'Subject Introduced',
        'removed': 'Subject Removed'
    },
    {
        'prefix': 'Mouse',
        'introduced': 'Mouse In',
        'removed': 'Mouse Out'
    }
]

experiment.group_extract_manual_annotations(bout_definitions,first_only=False)

Processing behaviors for p5-240826-091418...
⚠️ No bouts extracted for trial from /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core_csvs/p5-240826-091418.csv.
Processing behaviors for nn2-250117-085631...
⚠️ No bouts extracted for trial from /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core_csvs/nn2-250117-085631.csv.
Processing behaviors for p2-240523-081105...
⚠️ No bouts extracted for trial from /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core_csvs/p2-240523-081105.csv.
Processing behaviors for nn4-250118-094351...
⚠️ No bouts extracted for trial from /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/C1_2_3_Social_Pref/all_no_core_csvs/nn4-250118-094351.csv.
Processing behaviors for nn1-250117-081652...
⚠️ No bouts extracted for trial from /Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohort

## Long-Term Bar Graph - Total Investigation Time

In [ ]:
cups = "/Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/Social_Pref_Cup_Assignments.csv"

# Run this before plotting
fix_behavior_data_for_experiment(experiment, csv_base_path)

# Now all experiment.trials[...].behaviors are valid — plot will work!
df_with_pref, pref_list, no_pref_list = plot_custom_sniff_cup_assignments(
    experiment=experiment,
    assignment_csv_path=cups,  # Replace with actual path
    bar_color='#cccccc',
    figsize=(6, 8),
    pad_inches=0.1,
    title="Bimodal Exploration: Long-Term Cup Investigation Time",
    save=False,
    save_name="LongTerm_BarGraph.png"
)

# View results
print("Subjects Pref:", pref_list)
print("Subjects No_Pref:", no_pref_list)

### With colors

In [ ]:
plot_grouped_sniff_cup_assignments(
    experiment=experiment,
    assignment_csv_path=cups,
    bar_color='#cccccc',
    color_nn_pp='red',
    color_n_p='blue',
    figsize=(6, 8),
    title="Cohort Comp: Long Term Cup Investigation",
    pad_inches=0.1,
    save=False,
    save_name="CohortComp_LT_IT.png"
)

## Bimodal Behavior

In [ ]:
# Correct data input
trial_data_with_ids = get_trial_dataframes_with_ids(experiment)

# Create metadata with mapping
metadata_df = create_metadata_dataframe_with_agent_mapping(trial_data_with_ids, sniff_cup_csv_path=cups)

metadata_df

In [ ]:
# Trim pref_list to match the Subject index
trimmed_pref_list = [s.split('-')[0].lower() for s in pref_list]

plot_investigation_by_agent(
    df=metadata_df,                     # Your metadata dataframe
    subjects_to_include=trimmed_pref_list,     # List of Pref subject IDs (full trial names, e.g., 'p5-240826-091418')
    metric="Avg",                    # Choose "Total" or "Avg"
    title="High LT Preference (All Mice): Average Bout Duration",
    ylabel="Investigation Time (s)",
    xlabel="Agent",
    custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"],
    custom_xtick_colors=["black", "blue", "purple", "orange"],          # Optional: match labels to colors
    ylim=(0, 10),                     # Set Y-axis limits if desired
    bar_fill_color='white',
    bar_edge_color='black',
    bar_linewidth=3,
    bar_hatch='/',
    yticks_increment=5,               # Optional: Y-tick step
    figsize=(14, 8),
    pad_inches=0.1,
    save=True,
    save_name="Pref_ABD.png"
)

# pref_list, no_pref_list

## Dopamine Stuff (Need help with this)

In [ ]:
def build_da_metrics_dataframe(experiment, sniff_cup_csv_path, metric_list=None):
    """
    Builds a combined DataFrame of DA metrics across all trials with mapped 'Agent' labels.
    
    Parameters:
        - experiment: Experiment object after compute_all_da_metrics.
        - sniff_cup_csv_path (str): Path to CSV with sniff cup → agent assignments.
        - metric_list (list or None): If provided, include only these DA metrics in final output.
    
    Returns:
        - combined_df (pd.DataFrame): Rows of all DA metrics with Subject, Trial, Agent labels.
    """
    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].str.lower()

    # --- Subject → sniff cup → agent mapping ---
    subject_cup_map = {}
    for _, row in assign_df.iterrows():
        subject = row['Subject']
        subject_cup_map[subject] = {}
        for col in row.index:
            if col.lower().startswith('sniff cup'):
                cup_number = col.lower().replace('sniff cup ', '').strip()
                agent = str(row[col]).strip().lower()
                subject_cup_map[subject][f'sniff cup {cup_number}'] = agent

    # --- Collect all DA metrics with Agent + Trial labels ---
    all_rows = []

    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            print(f"⚠️ Trial '{trial_name}' has empty or missing behaviors. Skipping.")
            continue

        subject_prefix = trial_name.split('-')[0].lower()

        if subject_prefix not in subject_cup_map:
            print(f"⚠️ Subject '{subject_prefix}' not found in sniff cup CSV. Skipping.")
            continue

        cup_map = subject_cup_map[subject_prefix]
        df_behaviors = trial.behaviors.copy()

        if 'Behavior' not in df_behaviors.columns:
            print(f"⚠️ Trial '{trial_name}' missing 'Behavior' column. Skipping.")
            continue

        # Assign agent labels to each row
        df_behaviors["Agent"] = df_behaviors["Behavior"].str.lower().map(cup_map).fillna("unknown")
        df_behaviors["Trial"] = trial_name
        df_behaviors["Subject"] = subject_prefix

        # Include only selected metrics if provided
        keep_cols = ['Subject', 'Trial', 'Agent']
        if metric_list:
            metric_cols = [col for col in metric_list if col in df_behaviors.columns]
        else:
            metric_cols = [col for col in df_behaviors.columns 
                           if col not in ['Behavior', 'Trial', 'Subject', 'Agent']]
        
        keep_cols.extend(metric_cols)

        df_clean = df_behaviors[keep_cols]
        all_rows.append(df_clean)

    if not all_rows:
        print("⚠️ No valid data extracted. Returning empty DataFrame.")
        return pd.DataFrame()

    combined_df = pd.concat(all_rows, ignore_index=True)
    print(f"✅ Combined DA metric DataFrame created with {len(combined_df)} rows.")
    return combined_df


In [ ]:
# 1. Define flexible bout definitions
bout_definitions = [
    {
        'prefix': 'Subject',
        'introduced': ['Subject Introduced', 'Mouse In'],
        'removed': ['Subject Removed', 'Mouse Out']
    }
]

# 2. Path to sniff cup assignment CSV
cups = "/Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/Social_Pref_Cup_Assignments.csv"

# 3. Extract bouts
experiment.group_extract_manual_annotations_updated(
    bout_definitions=bout_definitions,
    first_only=True  # or False for all events per bout
)

# 4. Compute DA metrics
experiment.compute_all_da_metrics(
    max_bout_duration=5,
    use_adaptive=True,
    allow_bout_extension=True,
    mode='standard'
)

# 5. Build DA metric dataframe with agent labels
combined_df = build_da_metrics_dataframe(
    experiment,
    sniff_cup_csv_path=cups
)

print(combined_df.head())


In [ ]:
def extract_bouts_and_behaviors(self, csv_path, bout_definitions, first_only=True):
    """
    Updated to support both 'Time (s)' or 'Start (s)' + 'Duration (s)'.
    Also accepts lists for 'introduced' and 'removed' event labels.
    """
    import pandas as pd

    try:
        data = pd.read_csv(csv_path)
    except Exception as e:
        print(f"⚠️ Failed to load CSV for {self.name}: {e}")
        self.behaviors = pd.DataFrame()
        return

    if data.empty:
        print(f"⚠️ CSV {csv_path} is empty.")
        self.behaviors = pd.DataFrame()
        return

    # --- Time Extraction ---
    if 'Time (s)' in data.columns:
        data['Time'] = data['Time (s)']
    elif 'Start (s)' in data.columns:
        data['Time'] = data['Start (s)']
    else:
        print(f"⚠️ No time column found in CSV {csv_path}. Skipping.")
        self.behaviors = pd.DataFrame()
        return

    # --- Validate Required Columns ---
    required_cols = ['Behavior', 'Time']
    for col in required_cols:
        if col not in data.columns:
            print(f"⚠️ Required column '{col}' missing in {csv_path}. Skipping.")
            self.behaviors = pd.DataFrame()
            return

    # --- Initialize Behaviors DataFrame ---
    bout_rows = []

    for bout_def in bout_definitions:
        prefix = bout_def['prefix']
        introduced_labels = bout_def['introduced'] if isinstance(bout_def['introduced'], list) else [bout_def['introduced']]
        removed_labels = bout_def['removed'] if isinstance(bout_def['removed'], list) else [bout_def['removed']]

        # --- Find Start/Stop Times ---
        intro_events = data[data['Behavior'].isin(introduced_labels)].sort_values('Time')
        remove_events = data[data['Behavior'].isin(removed_labels)].sort_values('Time')

        if intro_events.empty or remove_events.empty:
            print(f"⚠️ No matching intro/remove events for bout '{prefix}' in {self.name}. Skipping this bout.")
            continue

        # --- Pair intro and removal events ---
        for idx, intro_row in intro_events.iterrows():
            intro_time = intro_row['Time']
            # Find the first removal event after this intro
            following_removes = remove_events[remove_events['Time'] > intro_time]
            if following_removes.empty:
                continue
            remove_time = following_removes.iloc[0]['Time']

            # Extract all events within this bout window
            bout_events = data[(data['Time'] >= intro_time) & (data['Time'] <= remove_time)].copy()
            bout_events['Bout'] = prefix
            bout_rows.append(bout_events)

            if first_only:
                break  # Only keep the first bout per definition

    if not bout_rows:
        print(f"⚠️ No bouts extracted for trial '{self.name}'.")
        self.behaviors = pd.DataFrame()
        return

    # --- Combine all bouts ---
    behaviors_df = pd.concat(bout_rows, ignore_index=True)

    # --- Duration ---
    if 'Duration (s)' not in behaviors_df.columns:
        print(f"⚠️ 'Duration (s)' column missing in {self.name}. Filling with NaN.")
        behaviors_df['Duration (s)'] = float('nan')

    self.behaviors = behaviors_df
    print(f"✅ Extracted {len(behaviors_df)} behaviors for {self.name}.")

experiment.group_extract_manual_annotations(
    bout_definitions=bout_definitions,
    first_only=True  # or True if you only want first events per bout
)

In [ ]:
cups = "/Users/naylajimenez/Downloads/papers/dopamine/cohort 1 + 2/all cohorts/Social_Pref_Cup_Assignments.csv"

# Proceed with DA metric computation after all files are processed
experiment.compute_all_da_metrics(
    max_bout_duration=5,       # Set the maximum bout duration to 5 seconds
    use_adaptive=True,         # Enable adaptive peak-following window
    allow_bout_extension=True, # Allow extending the bout window if needed
    mode='standard'                 # Consider only the first investigation event per bout
)

In [ ]:
import pandas as pd

def build_da_metrics_dataframe(experiment, sniff_cup_csv_path, metric_list=None):
    """
    Builds a combined DataFrame of DA metrics across all trials with mapped 'Agent' labels.
    
    Parameters:
        - experiment: Experiment object after compute_all_da_metrics.
        - sniff_cup_csv_path (str): Path to CSV with sniff cup → agent assignments.
        - metric_list (list or None): If provided, include only these DA metrics in final output.
    
    Returns:
        - combined_df (pd.DataFrame): Rows of all DA metrics with Subject, Trial, Agent labels.
    """
    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].str.lower()

    # --- Subject → sniff cup → agent mapping ---
    subject_cup_map = {}
    for _, row in assign_df.iterrows():
        subject = row['Subject']
        subject_cup_map[subject] = {}
        for col in row.index:
            if col.lower().startswith('sniff cup'):
                cup_number = col.lower().replace('sniff cup ', '').strip()
                agent = str(row[col]).strip().lower()
                subject_cup_map[subject][f'sniff cup {cup_number}'] = agent

    # --- Collect all DA metrics with Agent + Trial labels ---
    all_rows = []

    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            print(f"⚠️ Trial '{trial_name}' has empty or missing behaviors. Skipping.")
            continue

        subject_prefix = trial_name.split('-')[0].lower()

        if subject_prefix not in subject_cup_map:
            print(f"⚠️ Subject '{subject_prefix}' not found in sniff cup CSV. Skipping.")
            continue

        cup_map = subject_cup_map[subject_prefix]
        df_behaviors = trial.behaviors.copy()

        if 'Behavior' not in df_behaviors.columns:
            print(f"⚠️ Trial '{trial_name}' missing 'Behavior' column. Skipping.")
            continue

        # Assign agent labels to each row
        df_behaviors["Agent"] = df_behaviors["Behavior"].str.lower().map(cup_map).fillna("unknown")
        df_behaviors["Trial"] = trial_name
        df_behaviors["Subject"] = subject_prefix

        # Include only selected metrics if provided
        keep_cols = ['Subject', 'Trial', 'Agent']
        if metric_list:
            metric_cols = [col for col in metric_list if col in df_behaviors.columns]
        else:
            metric_cols = [col for col in df_behaviors.columns 
                           if col not in ['Behavior', 'Trial', 'Subject', 'Agent']]
        
        keep_cols.extend(metric_cols)

        df_clean = df_behaviors[keep_cols]
        all_rows.append(df_clean)

    if not all_rows:
        print("⚠️ No valid data extracted. Returning empty DataFrame.")
        return pd.DataFrame()

    combined_df = pd.concat(all_rows, ignore_index=True)
    print(f"✅ Combined DA metric DataFrame created with {len(combined_df)} rows.")
    return combined_df

# Run this after compute_all_da_metrics
combined_df = build_da_metrics_dataframe(
    experiment, 
    sniff_cup_csv_path=cups
)

combined_df

In [ ]:
def plot_da_metrics_by_agent(experiment, 
                             sniff_cup_csv_path,
                             metric_name="Mean Z-score", 
                             title="DA Response by Social Agent", 
                             ylabel="DA Signal (Z-score)", 
                             xlabel="Social Agent", 
                             custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"], 
                             custom_xtick_colors=["teal", "blue", "purple", "orange"], 
                             pref_subjects=None,
                             ylim=None, 
                             bar_color="#00B7D7", 
                             bar_fill_color='white',
                             bar_edge_color='black',
                             bar_linewidth=3,
                             bar_hatch='/', 
                             yticks_increment=None, 
                             figsize=(14,8), 
                             pad_inches=0.1,
                             save=False, 
                             save_name=None):
    """
    Plots DA metrics by agent (mapped via sniff cup CSV) for each trial.
    Agents: 'nothing', 'short_term', 'long_term', 'novel'.
    Styled with bar aesthetics and colored subject markers.
    """

    # Step 1: Map sniff cup assignments to DA behaviors
    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].str.lower()

    # Subject → sniff cup # → agent mapping
    subject_cup_map = {}
    for _, row in assign_df.iterrows():
        subject = row['Subject']
        subject_cup_map[subject] = {}
        for col in row.index:
            if col.lower().startswith('sniff cup'):
                cup_number = col.lower().replace('sniff cup ', '').strip()
                agent = str(row[col]).strip().lower()
                subject_cup_map[subject][f'sniff cup {cup_number}'] = agent

    # Assign agent labels to each row in each trial.behaviors
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            continue
        subject_prefix = trial_name.split('-')[0].lower()
        if subject_prefix not in subject_cup_map:
            print(f"Subject '{subject_prefix}' not found in sniff cup CSV.")
            continue
        cup_map = subject_cup_map[subject_prefix]
        df_behaviors = trial.behaviors.copy()
        df_behaviors["Agent"] = df_behaviors["Behavior"].str.lower().map(cup_map).fillna("unknown")
        trial.behaviors = df_behaviors  # update trial data

    # Step 2: Collect DA metric per agent per trial
    trial_data = []
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, "behaviors") or trial.behaviors.empty:
            continue

        subject_id = trial_name.split('-')[0].lower()
        if pref_subjects and subject_id not in [s.lower() for s in pref_subjects]:
            continue

        df = trial.behaviors.copy()
        if metric_name not in df.columns or 'Agent' not in df.columns:
            continue

        # Group by agent for each trial
        grouped = df.groupby("Agent", as_index=False)[metric_name].mean()
        grouped["Trial"] = trial_name
        trial_data.append(grouped)

    if not trial_data:
        print("⚠️ No DA data available to plot after filtering.")
        return

    combined_df = pd.concat(trial_data, ignore_index=True)

    # Focus on valid agents
    valid_agents = ['nothing', 'short_term', 'long_term', 'novel']
    combined_df = combined_df[combined_df["Agent"].isin(valid_agents)]

    if combined_df.empty:
        print("⚠️ No matching agent data found in combined DataFrame.")
        return

    # Pivot: trials as rows, agents as columns
    pivot_df = combined_df.pivot(index="Trial", columns="Agent", values=metric_name)

    # Ensure all agents present in pivot_df
    for agent in valid_agents:
        if agent not in pivot_df.columns:
            pivot_df[agent] = np.nan
    pivot_df = pivot_df[valid_agents]

    # Compute Mean & SEM per agent
    stats_df = combined_df.groupby("Agent")[metric_name].agg(['mean', 'sem']).reindex(valid_agents).reset_index()

    # Paired T-tests
    def perform_t_tests(pivot_df):
        comparisons = {
            "nothing_vs_short_term": ("nothing", "short_term"),
            "nothing_vs_long_term": ("nothing", "long_term"),
            "nothing_vs_novel": ("nothing", "novel")
        }
        results = {}
        for key, (a1, a2) in comparisons.items():
            if a1 in pivot_df.columns and a2 in pivot_df.columns:
                paired = pivot_df[[a1, a2]].dropna()
                if len(paired) > 1:
                    t_stat, p_value = ttest_rel(paired[a1], paired[a2])
                    results[key] = {"t_stat": t_stat, "p_value": p_value}
        return results

    t_test_results = perform_t_tests(pivot_df)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=figsize)

    # Bar plot with custom aesthetics
    ax.bar(valid_agents, stats_df["mean"], yerr=stats_df["sem"],
           capsize=6, color=bar_fill_color, edgecolor=bar_edge_color,
           linewidth=bar_linewidth, width=0.6, hatch=bar_hatch,
           error_kw=dict(elinewidth=3, capthick=3, zorder=5))

    # --- Lines + Colored Markers (No Borders) ---
    for trial in pivot_df.index:
        subject_prefix = trial.split('-')[0].lower()
        marker_color = '#15616F' if subject_prefix.startswith('n') else '#FFAF00' if subject_prefix.startswith('p') else 'gray'
        ax.plot(valid_agents, pivot_df.loc[trial], linestyle='-', color='gray', alpha=0.5, linewidth=2.5, zorder=1)
        ax.scatter(valid_agents, pivot_df.loc[trial], color=marker_color,
                   s=120, alpha=1.0, zorder=1)

    # --- Labels ---
    ax.set_ylabel(ylabel, fontsize=30, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=30, labelpad=12)
    ax.set_title(title, fontsize=28)

    # --- X-ticks ---
    ax.set_xticks(np.arange(len(valid_agents)))
    ax.set_xticklabels(custom_xtick_labels, fontsize=28)
    for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
        tick.set_color(color)

    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # --- Y-limits ---
    all_vals = np.concatenate([pivot_df.values.flatten(), stats_df["mean"].values])
    if ylim is None:
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)
    else:
        ax.set_ylim(ylim)

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    # --- Aesthetics ---
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    # --- Legend ---
    legend_elements = [
        Line2D([0], [0], marker='o', color='none', label='NAc',
               markerfacecolor='#15616F', markersize=12, markeredgewidth=0),
        Line2D([0], [0], marker='o', color='none', label='mPFC',
               markerfacecolor='#FFAF00', markersize=12, markeredgewidth=0)
    ]
    ax.legend(handles=legend_elements, title="Region", fontsize=20, title_fontsize=22,
              loc='upper right', frameon=True)

    # --- Significance ---
    if t_test_results:
        max_y = ax.get_ylim()[1]
        sig_y_offset = max_y * 0.05
        comparisons = {
            "nothing_vs_short_term": (0, 1),
            "nothing_vs_long_term": (0, 2),
            "nothing_vs_novel": (0, 3)
        }
        line_spacing = sig_y_offset * 2.5
        current_y = np.nanmax(stats_df["mean"]) + sig_y_offset

        for key, (x1, x2) in comparisons.items():
            if key in t_test_results:
                p_value = t_test_results[key]["p_value"]
                if p_value < 0.05:
                    significance = "**" if p_value < 0.01 else "*"
                    ax.plot([x1, x2], [current_y, current_y], color='black', linewidth=5)
                    ax.text((x1 + x2) / 2, current_y + sig_y_offset / 1.5, significance,
                            fontsize=40, ha='center', color='black')
                    current_y += line_spacing

    plt.tight_layout(pad=pad_inches)
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)

    plt.show()

# Trim pref_list to match the Subject index
trimmed_pref_list = [s.split('-')[0].lower() for s in no_pref_list]

plot_da_metrics_by_agent(experiment, 
                             metric_name="Max Peak", 
                             sniff_cup_csv_path = cups,
                             title = "DA Response by Social Agent", 
                             ylabel="DA Signal (Z-score)", 
                             xlabel="Social Agent", 
                             custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"], 
                             custom_xtick_colors=["black", "blue", "purple", "orange"], 
                             pref_subjects=trimmed_pref_list,
                             ylim=None, 
                             bar_color="#00B7D7", 
                             yticks_increment=None, 
                             figsize=(14,8), 
                             pad_inches=0.1,
                             save=False, 
                             save_name=None
                        )

In [ ]:
def plot_da_metrics_by_agent(experiment, 
                             sniff_cup_csv_path,
                             metric_name="Mean Z-score", 
                             title="DA Response by Social Agent", 
                             ylabel="DA Signal (Z-score)", 
                             xlabel="Social Agent", 
                             custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"], 
                             custom_xtick_colors=["teal", "blue", "purple", "orange"], 
                             pref_subjects=None,
                             ylim=None, 
                             bar_color="#00B7D7", 
                             bar_fill_color='white',
                             bar_edge_color='black',
                             bar_linewidth=3,
                             bar_hatch='/', 
                             yticks_increment=None, 
                             figsize=(14,8), 
                             pad_inches=0.1,
                             save=False, 
                             save_name=None):
    """
    Plots DA metrics by agent (mapped via sniff cup CSV) for each trial.
    Agents: 'nothing', 'short_term', 'long_term', 'novel'.
    Styled with bar aesthetics and colored subject markers.
    """
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D
    from scipy.stats import ttest_rel

    # --- Step 1: Map sniff cup assignments to DA behaviors ---
    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].str.lower()

    subject_cup_map = {}
    for _, row in assign_df.iterrows():
        subject = row['Subject']
        subject_cup_map[subject] = {}
        for col in row.index:
            if col.lower().startswith('sniff cup'):
                cup_number = col.lower().replace('sniff cup ', '').strip()
                agent = str(row[col]).strip().lower()
                subject_cup_map[subject][f'sniff cup {cup_number}'] = agent

    # Assign agents to each trial
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            continue
        subject_prefix = trial_name.split('-')[0].lower()
        if subject_prefix not in subject_cup_map:
            continue
        cup_map = subject_cup_map[subject_prefix]
        df_behaviors = trial.behaviors.copy()
        df_behaviors["Agent"] = df_behaviors["Behavior"].str.lower().map(cup_map).fillna("unknown")
        trial.behaviors = df_behaviors

    # --- Debug Check: Show which trials are skipped or included ---
    for trial_name in experiment.trials:
        if not hasattr(experiment.trials[trial_name], "behaviors") or experiment.trials[trial_name].behaviors.empty:
            print(f"Trial {trial_name} skipped: no behaviors.")
            continue

        if pref_subjects and trial_name.lower() not in [s.lower() for s in pref_subjects]:
            print(f"Trial {trial_name} skipped: not in pref_subjects.")
            continue

        print(f"✅ Including trial: {trial_name}")


    # --- Step 2: Collect DA metric per agent per trial ---
    trial_data = []
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, "behaviors") or trial.behaviors.empty:
            continue

        # ✅ FIXED: Compare full trial names to pref_subjects
        if pref_subjects and trial_name.lower() not in [s.lower() for s in pref_subjects]:
            continue

        df = trial.behaviors.copy()
        if metric_name not in df.columns or 'Agent' not in df.columns:
            continue

        grouped = df.groupby("Agent", as_index=False)[metric_name].mean()
        grouped["Trial"] = trial_name
        trial_data.append(grouped)

    if not trial_data:
        print("⚠️ No DA data available to plot after filtering.")
        return

    combined_df = pd.concat(trial_data, ignore_index=True)

    valid_agents = ['nothing', 'short_term', 'long_term', 'novel']
    combined_df = combined_df[combined_df["Agent"].isin(valid_agents)]

    if combined_df.empty:
        print("⚠️ No matching agent data found in combined DataFrame.")
        return

    pivot_df = combined_df.pivot(index="Trial", columns="Agent", values=metric_name)
    for agent in valid_agents:
        if agent not in pivot_df.columns:
            pivot_df[agent] = np.nan
    pivot_df = pivot_df[valid_agents]

    stats_df = combined_df.groupby("Agent")[metric_name].agg(['mean', 'sem']).reindex(valid_agents).reset_index()

    # --- Paired T-tests ---
    def perform_t_tests(pivot_df):
        comparisons = {
            "nothing_vs_short_term": ("nothing", "short_term"),
            "nothing_vs_long_term": ("nothing", "long_term"),
            "nothing_vs_novel": ("nothing", "novel")
        }
        results = {}
        for key, (a1, a2) in comparisons.items():
            paired = pivot_df[[a1, a2]].dropna()
            if len(paired) > 1:
                t_stat, p_value = ttest_rel(paired[a1], paired[a2])
                results[key] = {"t_stat": t_stat, "p_value": p_value}
        return results

    t_test_results = perform_t_tests(pivot_df)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=figsize)

    ax.bar(valid_agents, stats_df["mean"], yerr=stats_df["sem"],
           capsize=6, color=bar_fill_color, edgecolor=bar_edge_color,
           linewidth=bar_linewidth, width=0.6, hatch=bar_hatch,
           error_kw=dict(elinewidth=3, capthick=3, zorder=5))

    # --- Lines + Colored Markers (No Borders) ---
    for trial in pivot_df.index:
        subject_prefix = trial.split('-')[0].lower()
        marker_color = '#15616F' if subject_prefix.startswith('n') else '#FFAF00' if subject_prefix.startswith('p') else 'gray'
        ax.plot(valid_agents, pivot_df.loc[trial], linestyle='-', color='gray', alpha=0.5, linewidth=2.5, zorder=1)
        ax.scatter(valid_agents, pivot_df.loc[trial], color=marker_color,
                   s=120, alpha=1.0, zorder=1)  # No border

    # --- Labels ---
    ax.set_ylabel(ylabel, fontsize=30, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=30, labelpad=12)
    ax.set_title(title, fontsize=28)

    ax.set_xticks(np.arange(len(valid_agents)))
    ax.set_xticklabels(custom_xtick_labels, fontsize=28)
    for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
        tick.set_color(color)

    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    all_vals = np.concatenate([pivot_df.values.flatten(), stats_df["mean"].values])
    if ylim is None:
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)
    else:
        ax.set_ylim(ylim)

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    # --- Legend ---
    legend_elements = [
        Line2D([0], [0], marker='o', color='none', label='NAc',
               markerfacecolor='#15616F', markersize=12, markeredgewidth=0),
        Line2D([0], [0], marker='o', color='none', label='mPFC',
               markerfacecolor='#FFAF00', markersize=12, markeredgewidth=0)
    ]
    ax.legend(handles=legend_elements, title="Region", fontsize=20, title_fontsize=22,
              loc='upper right', frameon=True)

    # --- Significance ---
    if t_test_results:
        max_y = ax.get_ylim()[1]
        sig_y_offset = max_y * 0.05
        comparisons = {
            "nothing_vs_short_term": (0, 1),
            "nothing_vs_long_term": (0, 2),
            "nothing_vs_novel": (0, 3)
        }
        line_spacing = sig_y_offset * 2.5
        current_y = np.nanmax(stats_df["mean"]) + sig_y_offset

        for key, (x1, x2) in comparisons.items():
            if key in t_test_results:
                p_value = t_test_results[key]["p_value"]
                if p_value < 0.05:
                    significance = "**" if p_value < 0.01 else "*"
                    ax.plot([x1, x2], [current_y, current_y], color='black', linewidth=5)
                    ax.text((x1 + x2) / 2, current_y + sig_y_offset / 1.5, significance,
                            fontsize=40, ha='center', color='black')
                    current_y += line_spacing

    plt.tight_layout(pad=pad_inches)
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)

    plt.show()

plot_da_metrics_by_agent(experiment, 
                             sniff_cup_csv_path= cups,
                             metric_name="Max Peak", 
                             title="DA Response by Social Agent", 
                             ylabel="DA Signal (Z-score)", 
                             xlabel="Social Agent", 
                             custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"], 
                             custom_xtick_colors=["black", "blue", "purple", "orange"], 
                             pref_subjects=pref_list,
                             ylim=None, 
                             bar_color='white', 
                             bar_fill_color='white',
                             bar_edge_color='black',
                             bar_linewidth=3,
                             bar_hatch='/', 
                             yticks_increment=None, 
                             figsize=(14,8), 
                             pad_inches=0.1,
                             save=False, 
                             save_name=None)


In [ ]:
print("pref_subjects =", pref_list)


In [ ]:
def plot_da_metrics_by_agent(experiment, 
                             sniff_cup_csv_path,
                             metric_name="Mean Z-score", 
                             title="DA Response by Social Agent", 
                             ylabel="DA Signal (Z-score)", 
                             xlabel="Social Agent", 
                             custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"], 
                             custom_xtick_colors=["teal", "blue", "purple", "orange"], 
                             pref_subjects=None,
                             ylim=None, 
                             bar_color="#00B7D7", 
                             bar_fill_color='white',
                             bar_edge_color='black',
                             bar_linewidth=3,
                             bar_hatch='/', 
                             yticks_increment=None, 
                             figsize=(14,8), 
                             pad_inches=0.1,
                             save=False, 
                             save_name=None):
    """
    Plots DA metrics by agent (mapped via sniff cup CSV) for each trial.
    Agents: 'nothing', 'short_term', 'long_term', 'novel'.
    Styled with bar aesthetics and colored subject markers.
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D
    from scipy.stats import ttest_rel

    # Step 1: Map sniff cup assignments to DA behaviors
    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].str.lower()

    # Subject → sniff cup # → agent mapping
    subject_cup_map = {}
    for _, row in assign_df.iterrows():
        subject = row['Subject']
        subject_cup_map[subject] = {}
        for col in row.index:
            if col.lower().startswith('sniff cup'):
                cup_number = col.lower().replace('sniff cup ', '').strip()
                agent = str(row[col]).strip().lower()
                subject_cup_map[subject][f'sniff cup {cup_number}'] = agent

    # Assign agent labels to each row in each trial.behaviors
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            continue
        subject_prefix = trial_name.split('-')[0].lower()
        if subject_prefix not in subject_cup_map:
            continue
        cup_map = subject_cup_map[subject_prefix]
        df_behaviors = trial.behaviors.copy()
        df_behaviors["Agent"] = df_behaviors["Behavior"].str.lower().map(cup_map).fillna("unknown")
        trial.behaviors = df_behaviors  # update trial data

    # Step 2: Collect DA metric per agent per trial
    trial_data = []
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, "behaviors") or trial.behaviors.empty:
            continue

        trial_name_lc = trial_name.lower()
        if pref_subjects and trial_name_lc not in [s.lower() for s in pref_subjects]:
            continue  # NOW comparing full trial names

        df = trial.behaviors.copy()
        if metric_name not in df.columns or 'Agent' not in df.columns:
            continue

        grouped = df.groupby("Agent", as_index=False)[metric_name].mean()
        grouped["Trial"] = trial_name
        trial_data.append(grouped)

    if not trial_data:
        print("⚠️ No DA data available to plot after filtering.")
        return

    combined_df = pd.concat(trial_data, ignore_index=True)

    # Focus on valid agents
    valid_agents = ['nothing', 'short_term', 'long_term', 'novel']
    combined_df = combined_df[combined_df["Agent"].isin(valid_agents)]

    if combined_df.empty:
        print("⚠️ No matching agent data found in combined DataFrame.")
        return

    # Pivot: trials as rows, agents as columns
    pivot_df = combined_df.pivot(index="Trial", columns="Agent", values=metric_name)
    for agent in valid_agents:
        if agent not in pivot_df.columns:
            pivot_df[agent] = np.nan
    pivot_df = pivot_df[valid_agents]

    # Compute Mean & SEM per agent
    stats_df = combined_df.groupby("Agent")[metric_name].agg(['mean', 'sem']).reindex(valid_agents).reset_index()

    # Paired T-tests
    def perform_t_tests(pivot_df):
        comparisons = {
            "nothing_vs_short_term": ("nothing", "short_term"),
            "nothing_vs_long_term": ("nothing", "long_term"),
            "nothing_vs_novel": ("nothing", "novel")
        }
        results = {}
        for key, (a1, a2) in comparisons.items():
            if a1 in pivot_df.columns and a2 in pivot_df.columns:
                paired = pivot_df[[a1, a2]].dropna()
                if len(paired) > 1:
                    t_stat, p_value = ttest_rel(paired[a1], paired[a2])
                    results[key] = {"t_stat": t_stat, "p_value": p_value}
        return results

    t_test_results = perform_t_tests(pivot_df)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=figsize)

    # --- Bar Plot with Aesthetics ---
    ax.bar(valid_agents, stats_df["mean"], yerr=stats_df["sem"],
           capsize=6, color=bar_fill_color, edgecolor=bar_edge_color,
           linewidth=bar_linewidth, width=0.6, hatch=bar_hatch,
           error_kw=dict(elinewidth=3, capthick=3, zorder=5))

    # --- Lines + Colored Markers (No Borders) ---
    for trial in pivot_df.index:
        subject_prefix = trial.split('-')[0].lower()
        marker_color = '#15616F' if subject_prefix.startswith('n') else '#FFAF00' if subject_prefix.startswith('p') else 'gray'
        ax.plot(valid_agents, pivot_df.loc[trial], linestyle='-', color='gray', alpha=0.5, linewidth=2.5, zorder=1)
        ax.scatter(valid_agents, pivot_df.loc[trial], color=marker_color,
                   s=120, alpha=1.0, zorder=1)  # No edgecolor

    # --- Labels ---
    ax.set_ylabel(ylabel, fontsize=30, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=30, labelpad=12)
    ax.set_title(title, fontsize=28)

    # --- X-ticks ---
    ax.set_xticks(np.arange(len(valid_agents)))
    ax.set_xticklabels(custom_xtick_labels, fontsize=28)
    for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
        tick.set_color(color)

    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # --- Y-limits ---
    all_vals = np.concatenate([pivot_df.values.flatten(), stats_df["mean"].values])
    if ylim is None:
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)
    else:
        ax.set_ylim(ylim)

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    # --- Aesthetics ---
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    # --- Legend ---
    legend_elements = [
        Line2D([0], [0], marker='o', color='none', label='NAc',
               markerfacecolor='#15616F', markersize=12, markeredgewidth=0),
        Line2D([0], [0], marker='o', color='none', label='mPFC',
               markerfacecolor='#FFAF00', markersize=12, markeredgewidth=0)
    ]
    ax.legend(handles=legend_elements, title="Region", fontsize=20, title_fontsize=22,
              loc='upper right', frameon=True)

    # --- Significance ---
    if t_test_results:
        max_y = ax.get_ylim()[1]
        sig_y_offset = max_y * 0.05
        comparisons = {
            "nothing_vs_short_term": (0, 1),
            "nothing_vs_long_term": (0, 2),
            "nothing_vs_novel": (0, 3)
        }
        line_spacing = sig_y_offset * 2.5
        current_y = np.nanmax(stats_df["mean"]) + sig_y_offset

        for key, (x1, x2) in comparisons.items():
            if key in t_test_results:
                p_value = t_test_results[key]["p_value"]
                if p_value < 0.05:
                    significance = "**" if p_value < 0.01 else "*"
                    ax.plot([x1, x2], [current_y, current_y], color='black', linewidth=5)
                    ax.text((x1 + x2) / 2, current_y + sig_y_offset / 1.5, significance,
                            fontsize=40, ha='center', color='black')
                    current_y += line_spacing

    plt.tight_layout(pad=pad_inches)
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)

    plt.show()


plot_da_metrics_by_agent(
    experiment,
    sniff_cup_csv_path=cups,
    metric_name="Max Peak",
    title="DA Response by Social Agent",
    ylabel="DA Signal (Z-score)",
    xlabel="Social Agent",
    custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"],
    custom_xtick_colors=["black", "blue", "purple", "orange"],
    pref_subjects=pref_list
)


## Random Stuff